### 코로나로 인한 사람들의 흥미 변화 및 유망 품목을 파악하기 위해 앱 데이터를 크롤링해서 분석해보았습니다.

* 앱 순위를 크롤링한 사이트는 [모바일 인덱스](https://www.mobileindex.com/) 입니다.

먼저, 기간별로 어플 순위를 가져왔습니다.


이때, **구글 플레이 스토어(안드로이드)**와 **앱 스토어(IOS)**를 구분지어 데이터를 크롤링했습니다.  

**기간**은 **2019년 1월 ~ 12월**과 **2020년 1월 ~ 6월**의 데이터를 가져왔습니다.

### 기간별 앱 순위와 장르 가져오기

* 설정 초기화 및 필요한 모듈 임포트

In [42]:
from google.colab import drive
drive.mount('/content/gdrive')

!apt install chromium-chromedriver
!pip install selenium

from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

import warnings 
warnings.filterwarnings('ignore')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (83.0.4103.61-0ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [43]:
from bs4 import BeautifulSoup
import pandas as pd
import plotly.express as px
import numpy as np

먼저, 각 월말에 해당하는 데이터를 가져오기 위해 해당 사이트의 쿼리스트링을 활용하였습니다. 쿼리스트링을 변경하며 해당 날짜에 알맞은 데이터를 가져온 뒤, 해당 페이지의 소스를 받아옵니다.

이후 받아온 페이지 소스를 활용해 순위에 알맞은 앱 명과 장르를 가져오도록 함수로 구현하였습니다.

In [ ]:
genre_dic = {}

def makeUrl(dateList,i, kinds):
  if kinds == 'playstore': 
    url = 'https://www.mobileindex.com/app/get_rank_all?rt=r&mk=2&c=kr&t=app&rs=100&d=' + dateList[i]
  else:
    url = 'https://www.mobileindex.com/app/get_rank_all?rt=r&mk=1&c=kr&t=app&rs=100&d=' + dateList[i]
  return url

def getGenreUrl(url):
  driver = webdriver.Chrome('chromedriver',options=options)
  driver.get(url)

  html = driver.page_source
  soup = BeautifulSoup(html, 'html.parser')
  genreURL = soup.select('div.item-info > a')
  return genreURL

def get_genre(url,appname):
  if appname in genre_dic:
    return genre_dic[appname]
  dr = webdriver.Chrome('chromedriver',options=options)
  dr.get(url)
  ht = dr.page_source
  so = BeautifulSoup(ht, 'html.parser')
  genre = so.select('tr.text-center')[0].text
  index1 = genre.find('\n',1)
  if genre[index1+1] == '\n':
    genre_dic[appname] = '기타'
    return '기타'
  index2 = genre.find('\n',14)
  genre_dic[appname] = genre[(index1+1):index2]
  return genre[(index1+1):index2]
  
def separateAppRank(rank):
  free_rank = []
  pay_rank = []
  sales_rank = [] 
  for j in range(len(rank)):
    if len(rank[j]) == 100:
      free_rank.append(rank[j][0::2])
      pay_rank.append(rank[j][1::2])
    elif len(rank[j]) == 150 or len(rank[j]) == 300:
      free_rank.append(rank[j][0::3])
      pay_rank.append(rank[j][1::3])
      sales_rank.append(rank[j][2::3])
  return free_rank, pay_rank, sales_rank

def getAppRank(url, year):
  driver = webdriver.Chrome('chromedriver',options=options)
  driver.get(url)

  html = driver.page_source
  soup = BeautifulSoup(html, 'html.parser')
  notices = soup.select('span.appname')
  
  tmp = []
  for n in notices:
    tmp.append(n.text.strip())
  return tmp

위의 함수들을 활용하여, 연도와 기간, 그리고 스토어 종류별 알맞은 데이터 값들을 크롤링하였고, 이를 각각 list에 저장하였습니다.  

---

**함수 설명**
* makeUrl  
날짜와 스토어 종류, 날짜 인덱스 번호를 파라미터로 하여 알맞은 url를 만들어주는 함수

* getAppRank  
makeUrl를 통해 만든 url과 연도를 입력받아 해당 페이지의 순위별 어플명을 리스트에 저장하는 함수

* getGenreUrl  
makeUrl를 통해 만든 url를 이용하여 해당 태그에 속해있는 장르명을 순위 순서대로 받아오는 함수

* get_genre  
getGenreUrl로 만든 genre url를 활용하여 장르를 불러오는 함수. 클릭을 활용해야하기 때문에 selenium을 필수적으로 사용하였다.

* separateAppRank  
단순 크롤링을 통해 받아온 값들은 종류별 앱 순위가 아니므로 각 종류에 알맞은 값으로 분류하는 함수

---


위의 함수들을 활용하여 무료 앱은 무료 앱끼리, 유료는 유료끼리, 그리고 매출 순위까지 각각 합쳐 이후 데이터 분석을 위해 DataFrame으로 변환하는 과정을 거쳤습니다.

이 때 열 이름은 각 연도와 월 명을 언더바를 이용하여 19_1 과 같이 나타내었고, 장르는 19_1_genre 와 같이 나타내었습니다. 아래는 예시 코드의 일부입니다.

In [ ]:
col = []
for i in range(12):
  col.append('19_'+str(i+1))
  col.append('19_'+str(i+1)+'_gen')
for i in range(6):
  col.append('20_'+str(i+1))
  col.append('20_'+str(i+1)+'_gen')

appRank_free_playstore_df = pd.DataFrame(p_appRank_free)
appRank_free_playstore_df = appRank_free_playstore_df.T
appRank_free_playstore_df.columns = col
appRank_free_playstore_df.to_csv('appRank_free_playstore_2019.csv', encoding='utf-8-sig')

### 순위별 장르 점수 매기기 

데이터를 가져오는 작업이 끝났다면, 이제 데이터를 가공해야할 차례입니다.

사람들의 **흥미 변화 및 유망 품목을 파악하는 것**이 목표이기에, 각 앱 명이 아닌 앱의 장르를 기준으로 분석하였습니다.

순위별 점수를 장르에 매겨, 해당 장르의 합산 점수를 장르명과 점수의 딕셔너리 형태로 리스트에 저장합니다. 

이후, **2019년 1월 ~ 9월** 의 장르 점수 값들을 합산한 뒤 평균내어 **기준값**으로 설정하고, **2019년 10월부터 2020년 6월**의 장르 점수 값과, 기준값을 각각 비교합니다. 이 때 비교한 결과는 증감율로서 표현합니다.

2019년 10월의 데이터부터 증감율을 비교한 이유는 코로나 발생 기점인 2020년 1월을 기준으로 이전 3달의 변화 양상과 이후의 변화 양상을 확인하기 위해 10월부터 확인하였습니다.


* 먼저, 필요한 데이터를 불러옵니다. 이 csv 파일들은 모두 위의 장르 및 앱 순위 구하기를 통해 만든 데이터프레임을 csv로 저장한 파일입니다.

In [48]:
appRank_free_playstore_19_df = pd.read_csv('/content/gdrive/My Drive/dacon/app/appRank_free_playstore_2019.csv')
appRank_free_appstore_19_df = pd.read_csv('/content/gdrive/My Drive/dacon/app/appRank_free_appstore_2019.csv')
appRank_pay_playstore_19_df = pd.read_csv('/content/gdrive/My Drive/dacon/app/appRank_pay_playstore_2019.csv')
appRank_pay_appstore_19_df = pd.read_csv('/content/gdrive/My Drive/dacon/app/appRank_pay_appstore_2019.csv')
appRank_free_playstore_20_df = pd.read_csv('/content/gdrive/My Drive/dacon/app/appRank_free_playstore_2020.csv')
appRank_free_appstore_20_df = pd.read_csv('/content/gdrive/My Drive/dacon/app/appRank_free_appstore_2020.csv')
appRank_pay_playstore_20_df = pd.read_csv('/content/gdrive/My Drive/dacon/app/appRank_pay_playstore_2020.csv')
appRank_pay_appstore_20_df = pd.read_csv('/content/gdrive/My Drive/dacon/app/appRank_pay_appstore_2020.csv')
appRank_sales_playstore_20_df = pd.read_csv('/content/gdrive/My Drive/dacon/app/app_sales_rank_playstore_2020.csv')
appRank_sales_appstore_20_df = pd.read_csv('/content/gdrive/My Drive/dacon/app/app_sales_rank_appstore_2020.csv')
appRank_sales_playstore_19_df = pd.read_csv('/content/gdrive/My Drive/dacon/app/app_sales_rank_playstore_2019.csv')
appRank_sales_appstore_19_df = pd.read_csv('/content/gdrive/My Drive/dacon/app/app_sales_rank_appstore_2019.csv')

장르별 점수는 총 1~50위까지 있는 경우에 1위인 장르는 50점, 50위인 장르는 1점으로 할당하였습니다.

이후 각 장르 점수를 합산하여 '장르명':점수 와 같은 딕셔너리 형태로 나타내었으며 이를 월별 장르 점수로 묶은 뒤, 해당 연도의 리스트에 추가하였습니다.

각 장르 점수가 담긴 연도별 리스트의 명은 다음과 같습니다.  
* free_p_gen_dic_19 / free_p_gen_dic_20
* pay_p_gen_dic_19 / pay_p_gen_dic_20
* sales_p_gen_dic_19 / sales_p_gen_dic_20
* free_a_gen_dic_19 / free_a_gen_dic_20
* pay_a_gen_dic_19 / pay_a_gen_dic_20
* sales_a_gen_dic_19 / sales_a_gen_dic_20

이후의 모든 변수명을 나타낼 때에도 위와 같은 형태로 나타내는데, 변수명 설정 규칙은 다음과 같습니다.

* 유료/ 무료: pay/ free
* 플레이스토어/ 앱스토어: p/ a or playstore/ appstore
* 연도명 : 19/ 20



#### 플레이스토어 무료

장르별 점수를 매긴 뒤, *free_p_gen_dic_19*를 이용하여 월별 장르의 점수가 담긴 *free_playstore_19*라는 데이터프레임을 생성하였습니다.

In [ ]:
# appRank_free_playstore_19_df.drop(['Unnamed: 0'],axis=1,inplace=True)
# tmp = [i for i in list(appRank_free_playstore_19_df) if 'gen' in i]
# free_p_gen = list(set(np.array(appRank_free_playstore_19_df[tmp]).reshape(len(tmp)*50,)))
# free_p_gen_dic_19 = []
# for i in tmp:
#   free_p_gen_dic_tmp = {key: 0 for key in dict.fromkeys(free_p_gen).keys()}
#   score = 50
#   for j in appRank_free_playstore_19_df.dropna()[i]:
#     free_p_gen_dic_tmp[j] = free_p_gen_dic_tmp[j]+score
#     score -= 1
#   free_p_gen_dic_19.append(free_p_gen_dic_tmp)

# col = []
# for i in range(12):
#   col.append('19_'+str(i+1))

In [49]:
free_playstore_19 = pd.DataFrame(free_p_gen_dic_19)
free_playstore_19 = free_playstore_19.T
free_playstore_19.columns = col
free_playstore_19 = free_playstore_19.drop([free_playstore_19.index[0]])
free_playstore_19.head()

,19_1,19_2,19_3,19_4,19_5,19_6,19_7,19_8,19_9,19_10,19_11,19_12
식음료,96,87,47,84,83,75,90,61,66,72,49,89
라이프스타일,0,0,22,2,51,49,35,59,18,0,20,0
사진,72,34,41,88,78,22,73,31,44,85,0,65
도구,165,311,228,225,174,106,136,157,185,134,203,88
여행 및 지역정보,59,88,69,120,90,104,117,92,108,108,147,158


아래는 기준값(이후 변수명 **base**)을 구하는 과정입니다. 19년 1~9월의 장르별 합산 값의 평균을 계산하여 average 열에 저장합니다. 

또한 20년 데이터의 장르명의 존재여부를 확인하기 위해 *free_p_standard_19*의 장르명을 genre라는 리스트에 저장합니다.

* free_p_standard_19  
19년 1~9월 기준값을 담은 Df

In [ ]:
# free_p_standard_19 = free_playstore_19[['19_1','19_2','19_3','19_4','19_5','19_6','19_7','19_8','19_9']]
# free_p_standard_19 = free_p_standard_19.reset_index()
# free_p_standard_19 = free_p_standard_19.set_index('index')

# free_p_standard_19['average'] = 0.000
# print(len(free_p_standard_19))
# for i in range(24):
#   total = 0
#   for j in range(0,9):
#     total += free_p_standard_19.iloc[i][j]
#   free_p_standard_19['average'][i] = total/9

In [46]:
free_p_standard_19 = free_p_standard_19[['average']]
free_p_standard_19 = free_p_standard_19.reset_index()
genre = list(free_p_standard_19['index'])
free_p_standard_19 = free_p_standard_19.set_index('index')
free_p_standard_19.head()

24


,average
index,
식음료,76.555556
라이프스타일,26.222222
사진,53.666667
도구,187.444444
여행 및 지역정보,94.111111


이제, *free_playstore_19*의 변수명으로 Df를 생성합니다. *free_playstore_19*는 앞서 월별 장르 점수로 가공되어있는 위의 *free_playstore_19*에서 19년 10~12월의 데이터만을 가져온 Df 입니다.  

19년도 10~12월의 데이터는 기준값과 비교해서 증감율을 계산해야하므로, 증감율 계산 과정을 거칩니다.  

증감율은 **(현재값 - 기준값) / 기준값**의 과정을 거쳐 계산하였습니다.  

증감율을 계산하면, 각 월을 나타내는 열에 해당 장르의 증감율이 저장될 수 있도록 나타내었으며, 이후 plot을 하기에 용이하도록 데이터프레임의 구조를 변경하여 *free_playstore_19_graph*에 저장하였습니다.  

이때 사용된 genre_list는 *free_playstore_19*의 장르명을 모아둔 리스트입니다.  

In [ ]:
# free_playstore_19 = free_playstore_19[['19_10','19_11','19_12']]

# col = []
# for i in range(10,13):
#   col.append(str(i)+'월')

# free_playstore_19 = free_playstore_19.reset_index()

# for i in range(len(col)):
#   free_playstore_19[col[i]] = 0.000

# for i in range(len(free_playstore_19)):
#   base = free_playstore_19['index'][i]
#   if base in genre:
#     idx = genre.index(base)
#     base = free_p_standard_19.iloc[idx]['average']
#   else:
#     base = 1.0
#   for j in range(2, 5):
#     if base == 0:
#       value = 0.0
#     else:
#       value = (float(free_playstore_19.iloc[i][j]) - base) / base
#     free_playstore_19[col[j-2]][i] = value

# free_playstore_19 = free_playstore_19.set_index('index')
# free_playstore_19 = free_playstore_19[['10월','11월','12월']]
# free_playstore_19 = free_playstore_19.reset_index()
# free_playstore_19.columns = ['genre', '10월','11월','12월']
# month = list()
# for i in range(10,13):
#   month.append(list(free_playstore_19[str(i)+'월']))
# genre_list = list(free_playstore_19['genre'])

In [50]:
free_playstore_19_graph = pd.DataFrame()
for i in range(3):
  free_playstore_19_fig = pd.DataFrame(genre_list,columns = ['genre'])
  free_playstore_19_fig['value'] = month[i]
  free_playstore_19_fig['month'] = str(i+10)+'월'
  free_playstore_19_fig = free_playstore_19_fig[['genre','month','value']]
  free_playstore_19_graph = pd.concat([free_playstore_19_graph,free_playstore_19_fig], axis = 0)

free_playstore_19_graph.head()

,genre,month,value
0,식음료,10월,-0.359942
1,라이프스타일,10월,-0.237288
2,사진,10월,-1.000000
3,도구,10월,0.082988
4,여행 및 지역정보,10월,0.561983


20년의 데이터도 위와 같은 과정을 거쳐, 최종적으로 *free_playstore_20_graph*의 Df로 나타내었습니다. 

* 과정이 위와 같으므로 코드는 간략하게 나타내었습니다.

In [ ]:
# appRank_free_playstore_20_df.drop(['Unnamed: 0'], axis = 1, inplace=True)

# tmp = [i for i in list(appRank_free_playstore_20_df) if 'gen' in i]
# free_p_gen = list(set(np.array(appRank_free_playstore_20_df[tmp]).reshape(len(tmp)*50,)))
# free_p_gen_dic_20 = []
# for i in tmp:
#   free_p_gen_dic_tmp = {key: 0 for key in dict.fromkeys(free_p_gen).keys()}
#   score = 50
#   for j in appRank_free_playstore_20_df.dropna()[i]:
#     free_p_gen_dic_tmp[j] = free_p_gen_dic_tmp[j]+score
#     score -= 1
#   free_p_gen_dic_20.append(free_p_gen_dic_tmp)

# col = []
# for i in range(6):
#   col.append('20_'+str(i+1))

# free_playstore_20 = pd.DataFrame(free_p_gen_dic_20)
# free_playstore_20 = free_playstore_20.T
# free_playstore_20.columns = col
# free_playstore_20 = free_playstore_20.drop([free_playstore_20.index[0]])

# col = []
# for i in range(1,7):
#   col.append(str(i)+'월')

# free_playstore_20 = free_playstore_20.reset_index()

In [51]:
free_playstore_20.head()

,index,20_1,20_2,20_3,20_4,20_5,20_6
0,라이프스타일,31,97,0,0,38,0
1,사진,0,0,12,0,0,56
2,도구,137,135,92,208,134,227
3,지도/내비게이션,65,0,9,17,45,82
4,도서/참고자료,53,36,56,40,46,40


In [ ]:
# for i in range(len(col)):
#   free_playstore_20[col[i]] = 0.000
# # 
# for i in range(len(free_playstore_20)):
#   base = free_playstore_20['index'][i]
#   if base in genre:
#     idx = genre.index(base)
#     base = free_p_standard_19.iloc[idx]['average']
#     if base == 0.0:
#       base = 1.0
#   else:
#     base = 1.0
#   for j in range(1, 7):
#     if base == 0:
#       value = 0.0
#     else:
#       value = (float(free_playstore_20.iloc[i][j]) - base) / base
#     free_playstore_20[col[j-1]][i] = value

# free_playstore_20 = free_playstore_20.set_index('index')
# free_playstore_20 = free_playstore_20[['1월','2월','3월','4월','5월','6월']]
# free_playstore_20 = free_playstore_20.reset_index()
# free_playstore_20.columns = ['genre', '1월','2월','3월','4월','5월','6월']
# month = list()
# for i in range(1,7):
#   month.append(list(free_playstore_20[str(i)+'월']))
# genre_list = list(free_playstore_20['genre'])

In [53]:
free_playstore_20_graph = pd.DataFrame()
for i in range(6):
  free_playstore_20_fig = pd.DataFrame(genre_list,columns = ['genre'])
  free_playstore_20_fig['value'] = month[i]
  free_playstore_20_fig['month'] = str(i+1)+'월'
  free_playstore_20_fig = free_playstore_20_fig[['genre','month','value']]
  free_playstore_20_graph = pd.concat([free_playstore_20_graph,free_playstore_20_fig], axis = 0)

free_playstore_20_graph.head()

,genre,month,value
0,라이프스타일,1월,0.182203
1,사진,1월,-1.000000
2,도구,1월,-0.269117
3,지도/내비게이션,1월,0.069470
4,도서/참고자료,1월,0.133017


19년과 20년 데이터의 모든 전처리가 끝나 생성된 *free_playstore_19_graph* 와 *free_playstore_20_graph*를 concat하여 하나의 데이터프레임인 free_playstore_19_20_graph로 나타냅니다. 

컬럼 설명
* genre : 장르명
* month : 19.10 ~ 20.6월의 각 월
* value : 증감율

In [54]:
free_playstore_19_20_graph = pd.concat([free_playstore_19_graph, free_playstore_20_graph], axis = 0)
free_playstore_19_20_graph.head()

,genre,month,value
0,식음료,10월,-0.359942
1,라이프스타일,10월,-0.237288
2,사진,10월,-1.000000
3,도구,10월,0.082988
4,여행 및 지역정보,10월,0.561983


plotly를 활용하여, 가장 변화를 알아보기 쉽도록 bar chart에 애니메이션을 적용하였습니다. 

시각화 결과는 다음과 같습니다.

In [55]:
df = free_playstore_19_20_graph

fig = px.bar(df, x="genre", y="value", color="genre",
  animation_frame="month", animation_group="genre", range_y=[min(free_playstore_19_20_graph['value'])-5, max(free_playstore_19_20_graph['value'])+5])
fig.show()

### 데이터 시각화 결론 (플레이스토어 무료)
2019년 10-12월의 경우에는 비즈니스와 건강/운동이 10월에 조금 증가하였다가 이후에는 감소하여 모든 장르들이 별 추세를 보이지 않습니다.
이후 2020년의 증감율 추이를 보았을 때에는 2-5월을 지나며 **의료, 건강 운동, 비즈니스, 자동차, 교육** 관련 어플들이 기준값 대비 증가하는 추세를 보였고, **비즈니스**는 6월에서도 꾸준히 증가한 모습을 보였습니다. 


---


같은 방법으로 **플레이스토어 유료, 플레이스토어 매출, 앱스토어 무료, 앱스토어 유료, 앱스토어 매출** 도 동일하게 시각화를 진행하였습니다.

이 때, 다른 과정과 다른 점은 **플레이스토어 매출**입니다. 플레이스토어의 매출 자료는 2019년 10월부터 2020년 6월까지의 자료밖에 존재하지 않았기 때문에 앞서 설정한 기준이 아닌, 2019년의 데이터를 기준값으로 설정하고 2020년의 데이터를 증감율로서 비교하여 시각화하였습니다.

다른 모든 과정은 같은 과정이므로 의미없는 길어짐을 방지하기 위해 데이터 프레임과 시각화 결과만 나타내었습니다.

#### 플레이스토어 유료

In [ ]:
# appRank_pay_playstore_19_df.drop(['Unnamed: 0'], axis = 1, inplace= True)

# tmp = [i for i in list(appRank_pay_playstore_19_df) if 'gen' in i]
# pay_p_gen = list(set(np.array(appRank_pay_playstore_19_df[tmp]).reshape(len(tmp)*50,)))
# pay_p_gen_dic_19 = []
# for i in tmp:
#   pay_p_gen_dic_tmp = {key: 0 for key in dict.fromkeys(pay_p_gen).keys()}
#   score = 50
#   for j in appRank_pay_playstore_19_df.dropna()[i]:
#     pay_p_gen_dic_tmp[j] = pay_p_gen_dic_tmp[j]+score
#     score -= 1
#   pay_p_gen_dic_19.append(pay_p_gen_dic_tmp)

# col = []
# for i in range(12):
#   col.append('19_'+str(i+1))

# pay_playstore_19 = pd.DataFrame(pay_p_gen_dic_19)
# pay_playstore_19 = pay_playstore_19.T
# pay_playstore_19.columns = col
# pay_playstore_19 = pay_playstore_19.drop([pay_playstore_19.index[0]])

# pay_p_standard_19 = pay_playstore_19[['19_1','19_2','19_3','19_4','19_5','19_6','19_7','19_8','19_9']]
# pay_p_standard_19 = pay_p_standard_19.reset_index()
# pay_p_standard_19 = pay_p_standard_19.set_index('index')

# pay_p_standard_19['average'] = 0.000
# for i in range(len(pay_p_standard_19)):
#   total = 0
#   for j in range(0,9):
#     total += pay_p_standard_19.iloc[i][j]
#   pay_p_standard_19['average'][i] = total/9

# pay_p_standard_19 = pay_p_standard_19[['average']]
# pay_p_standard_19 = pay_p_standard_19.reset_index()
# genre = list(pay_p_standard_19['index'])
# pay_p_standard_19 = pay_p_standard_19.set_index('index')

# pay_playstore_19 = pay_playstore_19[['19_10','19_11','19_12']]

# col = []
# for i in range(10,13):
#   col.append(str(i)+'월')

# pay_playstore_19 = pay_playstore_19.reset_index()

# for i in range(len(col)):
#   pay_playstore_19[col[i]] = 0.000

# for i in range(len(pay_playstore_19)):
#   base = pay_playstore_19['index'][i]
#   if base in genre:
#     idx = genre.index(base)
#     base = pay_p_standard_19.iloc[idx]['average']
#   else:
#     base = 1.0
#   for j in range(2, 5):
#     if base == 0:
#       value = 0.0
#     else:
#       value = (float(pay_playstore_19.iloc[i][j]) - base) / base
#     pay_playstore_19[col[j-2]][i] = value

# pay_playstore_19 = pay_playstore_19.set_index('index')
# pay_playstore_19 = pay_playstore_19[['10월','11월','12월']]
# pay_playstore_19 = pay_playstore_19.reset_index()
# pay_playstore_19.columns = ['genre', '10월','11월','12월']
# month = list()
# for i in range(10,13):
#   month.append(list(pay_playstore_19[str(i)+'월']))
# genre_list = list(pay_playstore_19['genre'])

# pay_playstore_19_graph = pd.DataFrame()
# for i in range(3):
#   pay_playstore_19_fig = pd.DataFrame(genre_list,columns = ['genre'])
#   pay_playstore_19_fig['value'] = month[i]
#   pay_playstore_19_fig['month'] = str(i+10)+'월'
#   pay_playstore_19_fig = pay_playstore_19_fig[['genre','month','value']]
#   pay_playstore_19_graph = pd.concat([pay_playstore_19_graph,pay_playstore_19_fig], axis = 0)

# pay_playstore_19_graph.head()

# appRank_pay_playstore_20_df.drop(['Unnamed: 0'], axis = 1, inplace=True)

# tmp = [i for i in list(appRank_pay_playstore_20_df) if 'gen' in i]
# pay_p_gen = list(set(np.array(appRank_pay_playstore_20_df[tmp]).reshape(len(tmp)*50,)))
# pay_p_gen_dic_20 = []
# for i in tmp:
#   pay_p_gen_dic_tmp = {key: 0 for key in dict.fromkeys(pay_p_gen).keys()}
#   score = 50
#   for j in appRank_pay_playstore_20_df.dropna()[i]:
#     pay_p_gen_dic_tmp[j] = pay_p_gen_dic_tmp[j]+score
#     score -= 1
#   pay_p_gen_dic_20.append(pay_p_gen_dic_tmp)


# col = []
# for i in range(6):
#   col.append('20_'+str(i+1))

# pay_playstore_20 = pd.DataFrame(pay_p_gen_dic_20)
# pay_playstore_20 = pay_playstore_20.T
# pay_playstore_20.columns = col
# pay_playstore_20 = pay_playstore_20.drop([pay_playstore_20.index[0]])

# col = []
# for i in range(1,7):
#   col.append(str(i)+'월')

# pay_playstore_20 = pay_playstore_20.reset_index()

# for i in range(len(col)):
#   pay_playstore_20[col[i]] = 0.000

# for i in range(len(pay_playstore_20)):
#   base = pay_playstore_20['index'][i]
#   if base in genre:
#     idx = genre.index(base)
#     base = pay_p_standard_19.iloc[idx]['average']
#     if base == 0.0:
#       base = 1.0
#   else:
#     base = 1.0
#   for j in range(1, 7):
#     if base == 0:
#       value = 0.0
#     else:
#       value = (float(pay_playstore_20.iloc[i][j]) - base) / base
#     pay_playstore_20[col[j-1]][i] = value

# pay_playstore_20 = pay_playstore_20.set_index('index')
# pay_playstore_20 = pay_playstore_20[['1월','2월','3월','4월','5월','6월']]
# pay_playstore_20 = pay_playstore_20.reset_index()
# pay_playstore_20.head()


# month = list()
# for i in range(1,7):
#   month.append(list(pay_playstore_20[str(i)+'월']))

# genre_list = list(pay_playstore_20['index'])

# pay_playstore_20_graph = pd.DataFrame()
# for i in range(6):
#   pay_playstore_20_fig = pd.DataFrame(genre_list,columns = ['genre'])
#   pay_playstore_20_fig['value'] = month[i]
#   pay_playstore_20_fig['month'] = str(i+1)+'월'
#   pay_playstore_20_fig = pay_playstore_20_fig[['genre','month','value']]
#   pay_playstore_20_graph = pd.concat([pay_playstore_20_graph,pay_playstore_20_fig], axis = 0)

# pay_playstore_19_20_graph = pd.concat([pay_playstore_19_graph, pay_playstore_20_graph], axis = 0)

In [57]:
pay_playstore_19_20_graph.head()

,genre,month,value
0,도구,10월,-0.100592
1,사진,10월,-0.492272
2,맞춤 설정,10월,-0.558463
3,지도/내비게이션,10월,-1.000000
4,도서/참고자료,10월,-0.035714


In [58]:
df = pay_playstore_19_20_graph

fig = px.bar(df, x="genre", y="value", color="genre",
  animation_frame="month", animation_group="genre", range_y=[min(pay_playstore_19_20_graph['value'])-5, max(pay_playstore_19_20_graph['value'])+5])
fig.show()

### 데이터 시각화 결론 (플레이스토어 유료)
2019년 10-12월의 경우에는 모든 어플들이 별다른 증가 추세를 보이지 않습니다. 
이후 2020년의 증감율 추이를 보았을 때에는 1월부터 **여행 및 지역정보, 엔터테인먼트, 라이프스타일, 자동차**가 증가 추세를 보이기 시작하여 이후 **소셜, 예술/디자인** 또한 증가 추세를 보입니다.
유료 어플의 경우에는 무료 어플에 비해 시대적 상황에 따른 변화가 크지 않고, 개인의 취향 및 라이프스타일의 영향을 받는 경향이 있다는 것을 확인할 수 있습니다.

#### 플레이스토어 매출

In [59]:
# appRank_sales_playstore_19_df.drop(['Unnamed: 0'],axis=1,inplace= True)

# tmp = [i for i in list(appRank_sales_playstore_19_df) if 'gen' in i]
# sales_p_gen = list(set(np.array(appRank_sales_playstore_19_df[tmp]).reshape(len(tmp)*50,)))
# sales_p_gen_dic_19 = []
# for i in tmp:
#   sales_p_gen_dic_tmp = {key: 0 for key in dict.fromkeys(sales_p_gen).keys()}
#   score = 50
#   for j in appRank_sales_playstore_19_df.dropna()[i]:
#     sales_p_gen_dic_tmp[j] = sales_p_gen_dic_tmp[j]+score
#     score -= 1
#   sales_p_gen_dic_19.append(sales_p_gen_dic_tmp)

# col = []
# for i in range(6,12):
#   col.append('19_'+str(i+1))

# sales_playstore_19 = pd.DataFrame(sales_p_gen_dic_19)
# sales_playstore_19 = sales_playstore_19.T
# sales_playstore_19.columns = col
# sales_playstore_19 = sales_playstore_19.drop([sales_playstore_19.index[0]])
# sales_playstore_19 = sales_playstore_19.reset_index()
# sales_playstore_19 = sales_playstore_19.set_index('index')

# sales_playstore_19['average'] = 0.000

# for i in range(len(sales_playstore_19)):
#   total = 0
#   for j in range(0,6):
#     total += sales_playstore_19.iloc[i][j]
#   sales_playstore_19['average'][i] = total/9

# sales_playstore_19 = sales_playstore_19[['average']]
# sales_playstore_19 = sales_playstore_19.reset_index()
# genre = list(sales_playstore_19['index'])
# sales_playstore_19 = sales_playstore_19.set_index('index')
# sales_playstore_19.head()


# appRank_sales_playstore_20_df.drop(['Unnamed: 0'], axis = 1, inplace=True)
# tmp = [i for i in list(appRank_sales_playstore_20_df) if 'gen' in i]
# sales_p_gen = list(set(np.array(appRank_sales_playstore_20_df[tmp]).reshape(len(tmp)*50,)))
# sales_p_gen_dic_20 = []
# for i in tmp:
#   sales_p_gen_dic_tmp = {key: 0 for key in dict.fromkeys(sales_p_gen).keys()}
#   score = 50
#   for j in appRank_sales_playstore_20_df.dropna()[i]:
#     sales_p_gen_dic_tmp[j] = sales_p_gen_dic_tmp[j]+score
#     score -= 1
#   sales_p_gen_dic_20.append(sales_p_gen_dic_tmp)

# col = []
# for i in range(6):
#   col.append('20_'+str(i+1))

# sales_playstore_20 = pd.DataFrame(sales_p_gen_dic_20)
# sales_playstore_20 = sales_playstore_20.T
# sales_playstore_20.columns = col
# sales_playstore_20 = sales_playstore_20.drop([sales_playstore_20.index[0]])
# sales_playstore_20 = sales_playstore_20.reset_index()

# col = []
# for i in range(1,7):
#   col.append(str(i)+'월')

# for i in range(len(col)):
#   sales_playstore_20[col[i]] = 0.000

# for i in range(len(sales_playstore_20)):
#   base = sales_playstore_20['index'][i]
#   if base in genre:
#     idx = genre.index(base)
#     base = sales_playstore_19.iloc[idx]['average']
#     if base == 0.0:
#       base = 1.0
#   else:
#     base = 1.0
#   for j in range(1, 7):
#     if base == 0:
#       value = 0.0
#     else:
#       value = (float(sales_playstore_20.iloc[i][j]) - base) / base
#     sales_playstore_20[col[j-1]][i] = value

# sales_playstore_20 = sales_playstore_20.set_index('index')
# sales_playstore_20 = sales_playstore_20[['1월','2월','3월','4월','5월','6월']]
# sales_playstore_20 = sales_playstore_20.reset_index()
# sales_playstore_20.columns = ['genre', '1월','2월','3월','4월','5월','6월']


# month = list()
# for i in range(1,7):
#   month.append(list(sales_playstore_20[str(i)+'월']))
# genre_list = list(sales_playstore_20['genre'])

# sales_playstore_20_graph = pd.DataFrame()
# for i in range(6):
#   sales_playstore_20_fig = pd.DataFrame(genre_list,columns = ['genre'])
#   sales_playstore_20_fig['value'] = month[i]
#   sales_playstore_20_fig['month'] = str(i+1)+'월'
#   sales_playstore_20_fig = sales_playstore_20_fig[['genre','month','value']]
#   sales_playstore_20_graph = pd.concat([sales_playstore_20_graph,sales_playstore_20_fig], axis = 0)

In [60]:
sales_playstore_20_graph.head()

,genre,month,value
0,데이트,1월,0.550067
1,라이프스타일,1월,-0.013423
2,동영상 플레이어/편집기,1월,0.437500
3,스포츠,1월,0.704319
4,도서/참고자료,1월,0.363636


In [61]:
df = sales_playstore_20_graph

fig = px.bar(df, x="genre", y="value", color="genre",
  animation_frame="month", animation_group="genre", range_y=[min(sales_playstore_20_graph['value'])-5, max(sales_playstore_20_graph['value'])+5])
fig.show()

### 데이터 시각화 결론 (플레이스토어 매출)
2020년의 경우에는 **금융, 만화**가 초반 증가 추세를 보였습니다. 이후에도 **금융**은 꾸준히 증가 추세를 6월까지 보이는 것을 확인할 수 있습니다.


#### 앱스토어 무료

In [62]:
# appRank_free_appstore_19_df.drop(['Unnamed: 0'],axis=1,inplace=True)

# tmp = [i for i in list(appRank_free_appstore_19_df) if 'gen' in i]
# free_a_gen = list(set(np.array(appRank_free_appstore_19_df[tmp]).reshape(len(tmp)*100,)))
# free_a_gen_dic_19 = []
# for i in tmp:
#   free_a_gen_dic_tmp = {key: 0 for key in dict.fromkeys(free_a_gen).keys()}
#   score = 100
#   for j in appRank_free_appstore_19_df.dropna()[i]:
#     free_a_gen_dic_tmp[j] = free_a_gen_dic_tmp[j]+score
#     score -= 1
#   free_a_gen_dic_19.append(free_a_gen_dic_tmp)

# col = []
# for i in range(12):
#   col.append('19_'+str(i+1))

# free_appstore_19 = pd.DataFrame(free_a_gen_dic_19)
# free_appstore_19 = free_appstore_19.T
# free_appstore_19.columns = col
# free_appstore_19 = free_appstore_19.drop([free_appstore_19.index[0]])

# free_a_standard_19 = free_appstore_19[['19_1','19_2','19_3','19_4','19_5','19_6','19_7','19_8','19_9']]
# free_a_standard_19 = free_a_standard_19.reset_index()
# free_a_standard_19 = free_a_standard_19.set_index('index')
# free_a_standard_19.head()

# free_a_standard_19['average'] = 0.000
# for i in range(len(free_a_standard_19)):
#   total = 0
#   for j in range(0,9):
#     total += free_a_standard_19.iloc[i][j]
#   free_a_standard_19['average'][i] = total/9

# free_a_standard_19 = free_a_standard_19[['average']]
# free_a_standard_19 = free_a_standard_19.reset_index()
# genre = list(free_a_standard_19['index'])
# free_a_standard_19 = free_a_standard_19.set_index('index')

# free_appstore_19 = free_appstore_19[['19_10','19_11','19_12']]

# col = []
# for i in range(10,13):
#   col.append(str(i)+'월')

# free_appstore_19 = free_appstore_19.reset_index()

# for i in range(len(col)):
#   free_appstore_19[col[i]] = 0.000

# for i in range(len(free_appstore_19)):
#   base = free_appstore_19['index'][i]
#   if base in genre:
#     idx = genre.index(base)
#     base = free_a_standard_19.iloc[idx]['average']
#   else:
#     base = 1.0
#   for j in range(2, 5):
#     if base == 0:
#       value = 0.0
#     else:
#       value = (float(free_appstore_19.iloc[i][j]) - base) / base
#     free_appstore_19[col[j-2]][i] = value

# free_appstore_19 = free_appstore_19.set_index('index')
# free_appstore_19 = free_appstore_19[['10월','11월','12월']]
# free_appstore_19 = free_appstore_19.reset_index()
# free_appstore_19.columns = ['genre', '10월','11월','12월']
# month = list()
# for i in range(10,13):
#   month.append(list(free_appstore_19[str(i)+'월']))
# genre_list = list(free_appstore_19['genre'])

# free_appstore_19_graph = pd.DataFrame()
# for i in range(3):
#   free_appstore_19_fig = pd.DataFrame(genre_list,columns = ['genre'])
#   free_appstore_19_fig['value'] = month[i]
#   free_appstore_19_fig['month'] = str(i+10)+'월'
#   free_appstore_19_fig = free_appstore_19_fig[['genre','month','value']]
#   free_appstore_19_graph = pd.concat([free_appstore_19_graph,free_appstore_19_fig], axis = 0)

# appRank_free_appstore_20_df.drop(['Unnamed: 0'], axis = 1, inplace=True)

# tmp = [i for i in list(appRank_free_appstore_20_df) if 'gen' in i]
# free_a_gen = list(set(np.array(appRank_free_appstore_20_df[tmp]).reshape(len(tmp)*100,)))
# free_a_gen_dic_20 = []
# for i in tmp:
#   free_a_gen_dic_tmp = {key: 0 for key in dict.fromkeys(free_a_gen).keys()}
#   score = 100
#   for j in appRank_free_appstore_20_df.dropna()[i]:
#     free_a_gen_dic_tmp[j] = free_a_gen_dic_tmp[j]+score
#     score -= 1
#   free_a_gen_dic_20.append(free_a_gen_dic_tmp)

# col = []
# for i in range(6):
#   col.append('20_'+str(i+1))

# free_appstore_20 = pd.DataFrame(free_a_gen_dic_20)
# free_appstore_20 = free_appstore_20.T
# free_appstore_20.columns = col
# free_appstore_20 = free_appstore_20.drop([free_appstore_20.index[0]])
# free_appstore_20 = free_appstore_20.reset_index()

# col = []
# for i in range(1,7):
#   col.append(str(i)+'월')

# for i in range(len(col)):
#   free_appstore_20[col[i]] = 0.000

# for i in range(len(free_appstore_20)):
#   base = free_appstore_20['index'][i]
#   if base in genre:
#     idx = genre.index(base)
#     base = free_a_standard_19.iloc[idx]['average']
#     if base == 0.0:
#       base = 1.0
#   else:
#     base = 1.0
#   for j in range(1, 7):
#     if base == 0:
#       value = 0.0
#     else:
#       value = (float(free_appstore_20.iloc[i][j]) - base) / base
#     free_appstore_20[col[j-1]][i] = value

# free_appstore_20 = free_appstore_20.set_index('index')
# free_appstore_20 = free_appstore_20[['1월','2월','3월','4월','5월','6월']]
# free_appstore_20 = free_appstore_20.reset_index()
# free_appstore_20.columns = ['genre', '1월','2월','3월','4월','5월','6월']

# month = list()
# for i in range(1,7):
#   month.append(list(free_appstore_20[str(i)+'월']))

# genre_list = list(free_appstore_20['genre'])

# free_appstore_20_graph = pd.DataFrame()
# for i in range(6):
#   free_appstore_20_fig = pd.DataFrame(genre_list,columns = ['genre'])
#   free_appstore_20_fig['value'] = month[i]
#   free_appstore_20_fig['month'] = str(i+1)+'월'
#   free_appstore_20_fig = free_appstore_20_fig[['genre','month','value']]
#   free_appstore_20_graph = pd.concat([free_appstore_20_graph,free_appstore_20_fig], axis = 0)
#   free_appstore_19_20_graph = pd.concat([free_appstore_19_graph, free_appstore_20_graph], axis = 0)


In [63]:
free_appstore_19_20_graph.head()

,genre,month,value
0,식음료,10월,-0.158654
1,라이프스타일,10월,0.223029
2,Finance,10월,-1.000000
3,Navigation,10월,-1.000000
4,어드벤처,10월,0.061135


In [64]:
df = free_appstore_19_20_graph

fig = px.bar(df, x="genre", y="value", color="genre",
  animation_frame="month", animation_group="genre", range_y=[min(free_appstore_19_20_graph['value'])-5, max(free_appstore_19_20_graph['value'])+5])
fig.show()

### 데이터 시각화 결론 (앱스토어 무료)
2019년의 경우에는 별다른 변화가 없습니다. 2020년의 경우에는 **뉴스/잡지, 유틸리티, Navigation** 어플들이 주로 상승세를 보입니다. 4월 이후에는 **자동차 경주**게임 또한 상승 추세를 나타내었습니다.


#### 앱스토어 유료

In [65]:
# appRank_pay_appstore_19_df.drop(['Unnamed: 0'], axis = 1, inplace= True)

# tmp = [i for i in list(appRank_pay_appstore_19_df) if 'gen' in i]
# pay_a_gen = list(set(np.array(appRank_pay_appstore_19_df[tmp]).reshape(len(tmp)*100,)))
# pay_a_gen_dic_19 = []
# for i in tmp:
#   pay_a_gen_dic_tmp = {key: 0 for key in dict.fromkeys(pay_a_gen).keys()}
#   score = 100
#   for j in appRank_pay_appstore_19_df.dropna()[i]:
#     pay_a_gen_dic_tmp[j] = pay_a_gen_dic_tmp[j]+score
#     score -= 1
#   pay_a_gen_dic_19.append(pay_a_gen_dic_tmp)

# col = []
# for i in range(12):
#   col.append('19_'+str(i+1))

# pay_appstore_19 = pd.DataFrame(pay_a_gen_dic_19)
# pay_appstore_19 = pay_appstore_19.T
# pay_appstore_19.columns = col
# pay_appstore_19 = pay_appstore_19.drop([pay_appstore_19.index[0]])

# pay_a_standard_19 = pay_appstore_19[['19_1','19_2','19_3','19_4','19_5','19_6','19_7','19_8','19_9']]
# pay_a_standard_19 = pay_a_standard_19.reset_index()
# pay_a_standard_19 = pay_a_standard_19.set_index('index')

# pay_a_standard_19['average'] = 0.000
# for i in range(len(pay_a_standard_19)):
#   total = 0
#   for j in range(0,9):
#     total += pay_a_standard_19.iloc[i][j]
#   pay_a_standard_19['average'][i] = total/9

# pay_a_standard_19 = pay_a_standard_19[['average']]
# pay_a_standard_19 = pay_a_standard_19.reset_index()
# genre = list(pay_a_standard_19['index'])
# pay_a_standard_19 = pay_a_standard_19.set_index('index')
# pay_a_standard_19.head()

# pay_appstore_19 = pay_appstore_19[['19_10','19_11','19_12']]

# col = []
# for i in range(10,13):
#   col.append(str(i)+'월')

# pay_appstore_19 = pay_appstore_19.reset_index()

# for i in range(len(col)):
#   pay_appstore_19[col[i]] = 0.000

# for i in range(len(pay_appstore_19)):
#   base = pay_appstore_19['index'][i]
#   if base in genre:
#     idx = genre.index(base)
#     base = pay_a_standard_19.iloc[idx]['average']
#   else:
#     base = 1.0
#   for j in range(2, 5):
#     if base == 0:
#       value = 0.0
#     else:
#       value = (float(pay_appstore_19.iloc[i][j]) - base) / base
#     pay_appstore_19[col[j-2]][i] = value

# pay_appstore_19 = pay_appstore_19.set_index('index')
# pay_appstore_19 = pay_appstore_19[['10월','11월','12월']]
# pay_appstore_19 = pay_appstore_19.reset_index()
# pay_appstore_19.columns = ['genre', '10월','11월','12월']
# month = list()
# for i in range(10,13):
#   month.append(list(pay_appstore_19[str(i)+'월']))
# genre_list = list(pay_appstore_19['genre'])

# pay_appstore_19_graph = pd.DataFrame()
# for i in range(3):
#   pay_appstore_19_fig = pd.DataFrame(genre_list,columns = ['genre'])
#   pay_appstore_19_fig['value'] = month[i]
#   pay_appstore_19_fig['month'] = str(i+10)+'월'
#   pay_appstore_19_fig = pay_appstore_19_fig[['genre','month','value']]
#   pay_appstore_19_graph = pd.concat([pay_appstore_19_graph,pay_appstore_19_fig], axis = 0)

# appRank_pay_appstore_20_df.drop(['Unnamed: 0'], axis = 1, inplace=True)

# tmp = [i for i in list(appRank_pay_appstore_20_df) if 'gen' in i]
# pay_a_gen = list(set(np.array(appRank_pay_appstore_20_df[tmp]).reshape(len(tmp)*100,)))
# pay_a_gen_dic_20 = []
# for i in tmp:
#   pay_a_gen_dic_tmp = {key: 0 for key in dict.fromkeys(pay_a_gen).keys()}
#   score = 100
#   for j in appRank_pay_appstore_20_df.dropna()[i]:
#     pay_a_gen_dic_tmp[j] = pay_a_gen_dic_tmp[j]+score
#     score -= 1
#   pay_a_gen_dic_20.append(pay_a_gen_dic_tmp)


# col = []
# for i in range(6):
#   col.append('20_'+str(i+1))

# pay_appstore_20 = pd.DataFrame(pay_a_gen_dic_20)
# pay_appstore_20 = pay_appstore_20.T
# pay_appstore_20.columns = col
# pay_appstore_20 = pay_appstore_20.drop([pay_appstore_20.index[0]])
# pay_appstore_20.head()

# col = []
# for i in range(1,7):
#   col.append(str(i)+'월')

# pay_appstore_20 = pay_appstore_20.reset_index()


# for i in range(len(col)):
#   pay_appstore_20[col[i]] = 0.000

# for i in range(len(pay_appstore_20)):
#   base = pay_appstore_20['index'][i]
#   if base in genre:
#     idx = genre.index(base)
#     base = pay_a_standard_19.iloc[idx]['average']
#     if base == 0.0:
#       base = 1.0
#   else:
#     base = 1.0
#   for j in range(1, 7):
#     if base == 0:
#       value = 0.0
#     else:
#       value = (float(pay_appstore_20.iloc[i][j]) - base) / base
#     pay_appstore_20[col[j-1]][i] = value

# pay_appstore_20 = pay_appstore_20.set_index('index')
# pay_appstore_20 = pay_appstore_20[['1월','2월','3월','4월','5월','6월']]
# pay_appstore_20 = pay_appstore_20.reset_index()


# month = list()
# for i in range(1,7):
#   month.append(list(pay_appstore_20[str(i)+'월']))

# genre_list = list(pay_appstore_20['index'])

# pay_appstore_20_graph = pd.DataFrame()
# for i in range(6):
#   pay_appstore_20_fig = pd.DataFrame(genre_list,columns = ['genre'])
#   pay_appstore_20_fig['value'] = month[i]
#   pay_appstore_20_fig['month'] = str(i+1)+'월'
#   pay_appstore_20_fig = pay_appstore_20_fig[['genre','month','value']]
#   pay_appstore_20_graph = pd.concat([pay_appstore_20_graph,pay_appstore_20_fig], axis = 0)

# pay_appstore_19_20_graph = pd.concat([pay_appstore_19_graph,pay_appstore_20_graph], axis = 0)

In [66]:
pay_appstore_19_20_graph.head()

,genre,month,value
0,Education,10월,-1.000000
1,라이프스타일,10월,-1.000000
2,Finance,10월,-0.071749
3,Medical,10월,84.500000
4,음악/오디오,10월,-1.000000


In [67]:
df =pay_appstore_19_20_graph

fig = px.bar(df, x="genre", y="value", color="genre",
  animation_frame="month", animation_group="genre", range_y=[min(pay_appstore_19_20_graph['value'])-5, max(pay_appstore_19_20_graph['value'])+5])
fig.show()

### 데이터 시각화 결론 (앱스토어 유료)
2019년의 경우에는 **의료, 커뮤니케이션** 앱들의 큰 상승세가 나타납니다. 2020년에 들어서는 **커뮤니케이션, 도서** 등의 상승세를 확인할 수 있습니다.


#### 앱스토어 매출

In [68]:
# appRank_sales_appstore_19_df.drop(['Unnamed: 0'],axis=1,inplace= True)

# tmp = [i for i in list(appRank_sales_appstore_19_df) if 'gen' in i]
# sales_a_gen = list(set(np.array(appRank_sales_appstore_19_df[tmp]).reshape(len(tmp)*100,)))
# sales_a_gen_dic_19 = []
# for i in tmp:
#   sales_a_gen_dic_tmp = {key: 0 for key in dict.fromkeys(sales_a_gen).keys()}
#   score = 100
#   for j in appRank_sales_appstore_19_df.dropna()[i]:
#     sales_a_gen_dic_tmp[j] = sales_a_gen_dic_tmp[j]+score
#     score -= 1
#   sales_a_gen_dic_19.append(sales_a_gen_dic_tmp)

# col = []
# for i in range(12):
#   col.append('19_'+str(i+1))

# sales_appstore_19 = pd.DataFrame(sales_a_gen_dic_19)
# sales_appstore_19 = sales_appstore_19.T
# sales_appstore_19.columns = col
# sales_appstore_19 = sales_appstore_19.drop([sales_appstore_19.index[1]])
# sales_appstore_19 = sales_appstore_19.reset_index()
# sales_appstore_19 = sales_appstore_19.set_index('index')

# sales_a_standard_19 = sales_appstore_19[['19_1','19_2','19_3','19_4','19_5','19_6','19_7','19_8','19_9']]
# sales_a_standard_19 = sales_a_standard_19.reset_index()
# sales_a_standard_19 = sales_a_standard_19.set_index('index')

# sales_a_standard_19['average'] = 0.000

# for i in range(len(sales_a_standard_19)):
#   total = 0
#   for j in range(10):
#     total += sales_a_standard_19.iloc[i][j]
#   sales_a_standard_19['average'][i] = total/9

# sales_a_standard_19 = sales_a_standard_19[['average']]
# sales_a_standard_19 = sales_a_standard_19.reset_index()
# genre = list(sales_a_standard_19['index'])
# sales_a_standard_19 = sales_a_standard_19.set_index('index')

# sales_appstore_19 = sales_appstore_19[['19_10','19_11','19_12']]

# col = []
# for i in range(10,13):
#   col.append(str(i)+'월')

# sales_appstore_19 = sales_appstore_19.reset_index()

# for i in range(len(col)):
#   sales_appstore_19[col[i]] = 0.000

# for i in range(len(sales_appstore_19)):
#   base = sales_appstore_19['index'][i]
#   if base in genre:
#     idx = genre.index(base)
#     base = sales_a_standard_19.iloc[idx]['average']
#   else:
#     base = 1.0
#   for j in range(2, 5):
#     if base == 0:
#       value = 0.0
#     else:
#       value = (float(sales_appstore_19.iloc[i][j]) - base) / base
#     sales_appstore_19[col[j-2]][i] = value

# sales_appstore_19 = sales_appstore_19.set_index('index')
# sales_appstore_19 = sales_appstore_19[['10월','11월','12월']]
# sales_appstore_19 = sales_appstore_19.reset_index()
# sales_appstore_19.columns = ['genre', '10월','11월','12월']
# month = list()
# for i in range(10,13):
#   month.append(list(sales_appstore_19[str(i)+'월']))
# genre_list = list(sales_appstore_19['genre'])

# sales_appstore_19_graph = pd.DataFrame()
# for i in range(3):
#   sales_appstore_19_fig = pd.DataFrame(genre_list,columns = ['genre'])
#   sales_appstore_19_fig['value'] = month[i]
#   sales_appstore_19_fig['month'] = str(i+10)+'월'
#   sales_appstore_19_fig = sales_appstore_19_fig[['genre','month','value']]
#   sales_appstore_19_graph = pd.concat([sales_appstore_19_graph,sales_appstore_19_fig], axis = 0)

# appRank_sales_appstore_20_df.drop(['Unnamed: 0'], axis = 1, inplace=True)

# tmp = [i for i in list(appRank_sales_appstore_20_df) if 'gen' in i]
# sales_a_gen = list(set(np.array(appRank_sales_appstore_20_df[tmp]).reshape(len(tmp)*100,)))
# sales_a_gen_dic_20 = []
# for i in tmp:
#   sales_a_gen_dic_tmp = {key: 0 for key in dict.fromkeys(sales_a_gen).keys()}
#   score = 100
#   for j in appRank_sales_appstore_20_df.dropna()[i]:
#     sales_a_gen_dic_tmp[j] = sales_a_gen_dic_tmp[j]+score
#     score -= 1
#   sales_a_gen_dic_20.append(sales_a_gen_dic_tmp)

# col = []
# for i in range(6):
#   col.append('20_'+str(i+1))

# sales_appstore_20 = pd.DataFrame(sales_a_gen_dic_20)
# sales_appstore_20 = sales_appstore_20.T
# sales_appstore_20.columns = col
# sales_appstore_20 = sales_appstore_20.drop([sales_appstore_20.index[1]])
# sales_appstore_20 = sales_appstore_20.reset_index()

# col = []
# for i in range(1,7):
#   col.append(str(i)+'월')

# for i in range(len(col)):
#   sales_appstore_20[col[i]] = 0.000

# for i in range(len(sales_appstore_20)):
#   base = sales_appstore_20['index'][i]
#   if base in genre:
#     idx = genre.index(base)
#     base = sales_a_standard_19.iloc[idx]['average']
#     if base == 0.0:
#       base = 1.0
#   else:
#     base = 1.0
#   for j in range(1, 7):
#     if base == 0:
#       value = 0.0
#     else:
#       value = (float(sales_appstore_20.iloc[i][j]) - base) / base
#     sales_appstore_20[col[j-1]][i] = value

# sales_appstore_20 = sales_appstore_20.set_index('index')
# sales_appstore_20 = sales_appstore_20[['1월','2월','3월','4월','5월','6월']]
# sales_appstore_20 = sales_appstore_20.reset_index()
# sales_appstore_20.columns = ['genre', '1월','2월','3월','4월','5월','6월']

# month = list()
# for i in range(1,7):
#   month.append(list(sales_appstore_20[str(i)+'월']))
# genre_list = list(sales_appstore_20['genre'])

# sales_appstore_20_graph = pd.DataFrame()
# for i in range(6):
#   sales_appstore_20_fig = pd.DataFrame(genre_list,columns = ['genre'])
#   sales_appstore_20_fig['value'] = month[i]
#   sales_appstore_20_fig['month'] = str(i+1)+'월'
#   sales_appstore_20_fig = sales_appstore_20_fig[['genre','month','value']]
#   sales_appstore_20_graph = pd.concat([sales_appstore_20_graph,sales_appstore_20_fig], axis = 0)

# sales_appstore_19_20_graph = pd.concat([sales_appstore_19_graph, sales_appstore_20_graph], axis = 0)

In [71]:
sales_appstore_19_20_graph.head()

,genre,month,value
0,기타,10월,42.500000
1,어드벤처,10월,-0.953528
2,사진,10월,-1.000000
3,캐주얼 게임,10월,0.094845
4,롤플레잉,10월,0.120666


In [72]:
df = sales_appstore_19_20_graph

fig = px.bar(df, x="genre", y="value", color="genre",
  animation_frame="month", animation_group="genre", range_y=[min(sales_appstore_19_20_graph['value'])-5, max(sales_appstore_19_20_graph['value'])+5])
fig.show()

### 데이터 시각화 결론 (앱스토어 매출)
앱스토어 매출의 경우에는 **자동차 경주, 사진** 등의 몇가지 장르를 제외하고는 **기타** 장르의 점수 합이 컸으므로 별다른 비교가 어려웠습니다.


## 앱 데이터 시각화 결론

흥미 변화 및 유망 품목을 파악하기 위해 앱 데이터를 직접 크롤링해보고, 시각화를 통해 결과를 확인해보았습니다.

플레이스토어의 경우 코로나의 영향을 받았다고 할 수 있는 **자동차, 의료, 건강** 관련 앱들이 20년 이후 상승세를 나타내었습니다. 또한 무료/유료의 경우에 있어서는 무료 앱들이 시대적 상황을 더욱 반영한다는 것을 확인할 수 있었으며, 유료 앱은 개인의 취향에 따른 선호도가 더 크다는 것을 확인할 수 있었습니다.

포스트 코로나에 있어서는 **자동차, Navigation** 즉, 대중교통의 이용이 코로나 감염 위험을 높일 수 있으므로 개인 자차를 활용하게 되면서 자동차 관련 앱. 그리고 개인의 건강을 간단히 확인할 수 있는 **원격 의료 앱** 들이 유망하다는 것을 확인할 수 있습니다. 